In [ ]:
# Hàm để đặt một lệnh mua
import MetaTrader5 as mt5
from binance.client import Client
import os
# Đặt API key và Secret key
# api_key = "d9OIP56rF3aXhWMq9gmgIyn37uEw0X3D0GzWnm4Mh8fpL9J5bdXmJku4qwh7SHy7"
# api_secret = "HlUyOp8lzMyi3tZ0Q7oWpmrNF58dSbWgEVTApwsIZE6rJSoeS0BNsGZVwy4oEJGW"

# api_key = "9vFZgxlV2iruvGsBIATVTdONbPmN7RdV51Oxwycr8EHfkk2VTxTRLVwdp7Rz2KdM"
api_key = "y1tvZNa7rSq3VMTffUxFOqOWVzrf5Lbu66WlWjR7fw8RJSuNN1Z5JmQ4zBj64I9q"
# api_secret = "0iWhnDRvGmoOai8xhAhYwdu5bjZ3oCnV5TejQcdOppE96uGnOL2EvdHKfsPB08XJ"
api_secret = "fS5o1d86LUF3IxCtMh3RsHClTaWpDjVA5Ieb5L3y3hCxrg6yoBr7HUq0oIa3PFpV"

def orderSendCrypto(data):
    # Lấy dữ liệu từ từ điển
    datetime = data['Datetime']
    open_price = data['Open']
    close_price = data['Close']
    volume = data['Volume']
    sma = data['SMA']
    short_ema = data['short_ema']
    long_ema = data['long_ema']
    macd = data['MACD']
    signal_line = data['Signal_Line']
    buy_signal = data['Buy_Signal']
    
    sell_signal = data['Sell_Signal']
    symbol = data['Symbol']
    insertdate = data['Insertdate']
    
    # Khởi tạo client
    client = Client(api_key, api_secret)

    lot_quantity = 0.003

    order = None

    try:
        if buy_signal == 'True':  # Kiểm tra xem có phải là giá trị Boolean
            # Đặt lệnh mua
            order = client.order_market_buy(symbol='ETHUSDT', quantity=lot_quantity)
        elif sell_signal == 'True':
            # Kiểm tra số dư
            balance = client.get_asset_balance(asset='ETH')
            # Chỉ định số lượng dựa trên số dư
            lot_quantity = min(lot_quantity, float(balance['free']))
            # Đặt lệnh bán
            order = client.order_market_sell(symbol='ETHUSDT', quantity=lot_quantity)
    except Exception as e:
        print(f"An error occurred: {e}")

    print(order)


In [ ]:
import redis

def entryCrypto(hash):
    # Xử lý dữ liệu ở đây
    print("Dữ liệu đã được xử lý")

    # Tạo kết nối Redis
    r = redis.Redis(host='localhost', port=6379, db=0)

    # Hash key
    hash_key = hash

    # Đọc dữ liệu từ hash
    data = r.hgetall(hash_key)
    
    if data:
        # Chuyển đổi dữ liệu từ bytes sang chuỗi nếu cần
        data = {key.decode('utf-8'): value.decode('utf-8') for key, value in data.items()}
        # Xử lý dữ liệu
        orderSendCrypto(data)
        # Xóa hash sau khi xử lý
        r.delete(hash_key)
        print(f"Hash '{hash_key}' đã được xóa khỏi Redis.")
    else:
        print(f"Không tìm thấy dữ liệu cho hash '{hash_key}'.")

In [ ]:
from datetime import datetime, timedelta
import schedule
import time

def schedule_entryForex():
    entryCrypto('OG_CRTO_MACD dua tren HA')

# Lên lịch hàm scan_market để chạy mỗi 15 phút
schedule.every(10).seconds.do(schedule_entryForex)

while True:
    schedule.run_pending()  # Hàm này được gọi liên tục để kiểm tra xem có công việc nào đã đến thời gian cần thực hiện hay không
    time.sleep(10)